## basics

- https://github.com/microsoft/DeepSpeed
    - https://www.microsoft.com/en-us/research/blog/turing-nlg-a-17-billion-parameter-language-model-by-microsoft/
- pip 安装之后，可以通过 `ds_report` 命令查看环境配置信息；
- bag of tricks
    - $O(n^2)$（$n$ 表示序列长度） => sparse attention；
        - 10x longer seq，up to 6x faster；

## ZeRO: Zero Redundancy Optimizer

## 使用

### inference

- inference-tutorial
    - https://github.com/microsoft/DeepSpeed/blob/master/docs/_tutorials/inference-tutorial.md

In [1]:
import os
import torch
import transformers
from transformers import pipeline
import deepspeed

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To avoid warnings about parallelism in tokenizers

# distributed setup
local_rank = int(os.getenv("LOCAL_RANK", "0"))
world_size = int(os.getenv("WORLD_SIZE", "1"))
torch.cuda.set_device(local_rank)
deepspeed.init_distributed()

[2023-12-14 22:10:22,675] [INFO] [comm.py:606:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-12-14 22:10:23,210] [INFO] [comm.py:656:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=169.254.3.1, master_port=29500
[2023-12-14 22:10:23,211] [INFO] [comm.py:622:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [5]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [6]:
local_rank = int(os.getenv('LOCAL_RANK', '0'))
world_size = int(os.getenv('WORLD_SIZE', '1'))
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B',
                     device=local_rank)

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [7]:
generator.model = deepspeed.init_inference(generator.model,
                                           mp_size=world_size,
                                           dtype=torch.float,
                                           replace_with_kernel_inject=True)

string = generator("DeepSpeed is", do_sample=True, min_length=50)
if not torch.distributed.is_initialized() or torch.distributed.get_rank() == 0:
    print(string)

[2023-12-15 22:50:40,372] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.9.2, git-hash=unknown, git-branch=unknown
[2023-12-15 22:50:40,376] [WARNING] [config_utils.py:69:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2023-12-15 22:50:40,377] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /home/whaow/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Creating extension directory /home/whaow/.cache/torch_extensions/py310_cu117/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/whaow/.cache/torch_extensions/py310_cu117/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/9] /usr/local/cuda-11.7/bin/nvcc  -DTORCH_EXTENSION_NAME=transformer_inference -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/ops/csrc/transformer/inference/includes -I/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/ops/csrc/includes -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/TH -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda-11.7/include -isystem /home/whaow/anaconda3/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencod

[6/9] /usr/local/cuda-11.7/bin/nvcc  -DTORCH_EXTENSION_NAME=transformer_inference -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/ops/csrc/transformer/inference/includes -I/home/whaow/anaconda3/lib/python3.10/site-packages/deepspeed/ops/csrc/includes -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/TH -isystem /home/whaow/anaconda3/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda-11.7/include -isystem /home/whaow/anaconda3/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencod

[9/9] c++ pt_binding.o gelu.cuda.o relu.cuda.o layer_norm.cuda.o softmax.cuda.o dequantize.cuda.o apply_rotary_pos_emb.cuda.o transform.cuda.o -shared -lcurand -L/home/whaow/anaconda3/lib/python3.10/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda -ltorch -ltorch_python -L/usr/local/cuda-11.7/lib64 -lcudart -o transformer_inference.so
Time to load transformer_inference op: 25.59630537033081 seconds
[2023-12-15 22:51:06,713] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 2560, 'intermediate_size': 10240, 'heads': 20, 'num_hidden_layers': -1, 'fp16': False, 'pre_layer_norm': True, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'q_int8': False, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 256, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncTyp

Loading extension module transformer_inference...
Using /home/whaow/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module transformer_inference, skipping build step...
Loading extension module transformer_inference...
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


------------------------------------------------------
Free memory : 2.528137 (GigaBytes)  
Total memory: 23.649719 (GigaBytes)  
Requested memory: 0.800781 (GigaBytes) 
Setting maximum total tokens (input + output) to 1024 
WorkSpace: 0x7f5500000000 
------------------------------------------------------
[{'generated_text': 'DeepSpeed is not just a name for your hard drive. It’s a name that tells your computer about your system configuration and how it can improve storage performance. You can run one of these programs from the command line or right in the Task'}]
